In [17]:
import numpy as np

file = open('eng_jpn.txt', 'r', encoding='utf-8')
lines = file.readlines()
file.close()
print(len(lines))

62487


In [27]:
# shuffle and split
np.random.shuffle(lines)
train = lines[:int(len(lines)*0.8)]
val = lines[int(len(lines)*0.8):int(len(lines)*0.9)]
test = lines[int(len(lines)*0.9):]

print(len(train), len(val), len(test))

# write to files
train_file = open('train.txt', 'w', encoding='utf-8')
val_file = open('val.txt', 'w', encoding='utf-8')
test_file = open('test.txt', 'w', encoding='utf-8')

train_file.writelines(train)
val_file.writelines(val)
test_file.writelines(test)

train_file.close()
val_file.close()
test_file.close()

49989 6249 6249


In [38]:
train_file = open('./lstm_dataset/test.txt', 'r', encoding='utf-8') # train, val, test
# split into english and japanese
train_lines = train_file.readlines()
train_file.close()

train_jpn = []
train_eng = []
for line in train_lines:
    try:
        jpn, eng = line.strip().split('\t')
    except:
        print(line)
        continue
    train_jpn.append(jpn + '\n')
    train_eng.append(eng + '\n')

train_jpn_file = open('test_jpn.txt', 'w', encoding='utf-8')
train_eng_file = open('test_eng.txt', 'w', encoding='utf-8')

train_jpn_file.writelines(train_jpn)
train_eng_file.writelines(train_eng)

train_jpn_file.close()
train_eng_file.close()

In [79]:
import MeCab
from collections import defaultdict
import random

mecab = MeCab.Tagger('-Owakati')

def generate_cbow_pairs(text, window_size=2, pad_token='<pad>'):
    words = [pad_token] * window_size + text.strip().split() + [pad_token] * window_size
    data = []
    for i in range(window_size, len(words) - window_size):
        context = words[i-window_size:i] + words[i+1:i+window_size+1]
        target = words[i]
        data.append((context, target))
    return data

file_path = 'train_jpn.txt'
train_data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        processed_line = mecab.parse(line)
        cbow_pairs = generate_cbow_pairs(processed_line, window_size=2)
        train_data.extend(cbow_pairs)

random_samples = random.sample(train_data, 5)
for context, target in random_samples:
    print('Context:', context, 'Target:', target)


Context: ['ほしい', 'の', '<pad>', '<pad>'] Target: 。
Context: ['<pad>', '<pad>', '寒かっ', 'た'] Target: 外
Context: ['<pad>', '<pad>', 'は', '自分'] Target: 私
Context: ['<pad>', '<pad>', 'と', 'メアリー'] Target: トム
Context: ['本当', 'に', 'わ', 'ね'] Target: 細かい


In [80]:
train_cbow_file = open('./cbow_jpn_dataset/train_cbow.pkl', 'wb')

import pickle

pickle.dump(train_data, train_cbow_file)
train_cbow_file.close()

In [68]:
import torch

# 读取 CBOW 数据集
train_cbow_file = open('./cbow_jpn_dataset/test_cbow.pkl', 'rb')
train_data = pickle.load(train_cbow_file)
train_cbow_file.close()

for context, target in train_data[:5]:
    print('Context:', context, 'Target:', target)

Context: ['です', 'ね'] Target: 暑い
Context: ['暑い', 'ね', '。'] Target: です
Context: ['暑い', 'です', '。'] Target: ね
Context: ['です', 'ね'] Target: 。
Context: ['が', '一筋'] Target: 涙


In [87]:
import nltk
from nltk.tokenize import word_tokenize
import random

nltk.download('punkt')

def generate_cbow_pairs(text, window_size=2, pad_token='<pad>'):
    words = [pad_token] * window_size + word_tokenize(text) + [pad_token] * window_size
    data = []
    for i in range(window_size, len(words) - window_size):
        context = words[i-window_size:i] + words[i+1:i+window_size+1]
        target = words[i]
        data.append((context, target))
    return data

file_path = 'val_eng.txt'
train_data = []

with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        cbow_pairs = generate_cbow_pairs(line)
        train_data.extend(cbow_pairs)

# 打印一些随机的训练样本进行检查
random_samples = random.sample(train_data, 5)
for context, target in random_samples:
    print('Context:', context, 'Target:', target)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\forev\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Context: ['of', 'them', '<pad>', '<pad>'] Target: .
Context: ['All', 'the', 'got', 'seasick'] Target: passengers
Context: ['cold', '.', '<pad>', '<pad>'] Target: ''
Context: ['<pad>', '<pad>', 'is', 'mad'] Target: Tom
Context: ['left', 'behind', '<pad>', '<pad>'] Target: ?


In [88]:
train_cbow_file = open('./cbow_eng_dataset/val_cbow.pkl', 'wb')

import pickle

pickle.dump(train_data, train_cbow_file)
train_cbow_file.close()

In [60]:
import torch

# 读取 CBOW 数据集
train_cbow_file = open('./cbow_eng_dataset/train_cbow.pkl', 'rb')
train_data = pickle.load(train_cbow_file)
train_cbow_file.close()

for context, target in train_data[:5]:
    print('Context:', context, 'Target:', target)

Context: ['am', 'going'] Target: I
Context: ['I', 'going', 'to'] Target: am
Context: ['I', 'am', 'to', 'go'] Target: going
Context: ['am', 'going', 'go', 'to'] Target: to
Context: ['going', 'to', 'to', 'America'] Target: go
